In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn import metrics

Veride aslında eksiklik yoktur fakat evlerde bulunmayan öznitelikler Nan olarak kaydedilmiştir. Bu verilerin kolay işlenebilmesi adına "Nan" veriler "0" ile doldurulur.

In [2]:
df = pd.read_csv("/home/mete/Workspace/Okul/Ders/Bil479/Data/train.csv")
df_test = pd.read_csv("/home/mete/Workspace/Okul/Ders/Bil479/Data/test.csv") # hedef sütunu içermiyor
df_correct = pd.read_csv("/home/mete/Workspace/Okul/Ders/Bil479/Data/testtarget.csv")

In [3]:
df = df.fillna(0)
df_test = df_test.fillna(0)

Verinin coralasyon matrisi çıkartılır ve hedef (Satış fiyatı - SalePrice) ile ilişki oranı "0.2" den fazla olan
öznitelikler seçilir.

In [4]:
corr = df.corr()
high_correlated_columns = corr[abs(corr["SalePrice"])>0.2].index
high_correlated_columns = high_correlated_columns.drop("SalePrice")

Hedef öznitelik veriden ayrıştırılır ve Y olarak saklanır. Geri kalan özniteikler "MinMaxScaler" kullanılarak
normalize edilir. Normalize edilmiş öznitelik değerleri X olarak saklanır.

In [5]:
x_train = df[high_correlated_columns].values
x_test = df_test[high_correlated_columns].values

In [6]:
y_test = df_correct["SalePrice"]
y_train = df["SalePrice"]

In [7]:
y_train_max = max(y_train.values)
y_train_min = min(y_train.values)
y_train = (y_train-y_train_min)/(y_train_max-y_train_min)

In [8]:
y_test_max = max(y_test.values)
y_test_min = min(y_test.values)
y_test = (y_test-y_test_min)/(y_test_max-y_test_min)

In [9]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

x_train = min_max_scaler.fit_transform(x_train)
x_test = min_max_scaler.fit_transform(x_test)

In [10]:
model = tf.keras.Sequential([
      
      tf.keras.layers.Dense(256, activation='relu', input_shape=[x_train.shape[1]]),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dropout(0.05),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(16, activation='relu'),
      tf.keras.layers.Dense(1)
])

In [11]:
optimizer = tf.keras.optimizers.RMSprop(0.01)
model.compile(loss='mean_absolute_error',
            optimizer=optimizer,
            metrics=['mean_absolute_error'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               5888      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                5

In [13]:
EPOCHS = 250
history = model.fit(x_train, y_train,epochs=EPOCHS, verbose=0)

In [14]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(15)

,loss,mean_absolute_error,epoch
235,0.024963,0.024963,235
236,0.025632,0.025632,236
237,0.026785,0.026785,237
238,0.025444,0.025444,238
239,0.025370,0.025370,239
240,0.025482,0.025482,240
241,0.024720,0.024720,241
242,0.025370,0.025370,242
243,0.025921,0.025921,243
244,0.026775,0.026775,244


In [15]:
#Training Data

In [16]:
y_pred = model.predict(x_train)
metrics.r2_score(y_train,y_pred)

0.853850618226707

In [17]:
i = 200
print(f"actual={y_train[i]}, prediction={y_pred[i]}")

actual=0.1459519511179003, prediction=[0.13559389]


In [18]:
#Test Data

In [19]:
y_pred = model.predict(x_test)

In [20]:
i = 900
print(f"actual={y_test[i]}, prediction={y_pred[i]}")

actual=0.21954265199406853, prediction=[0.19379526]


In [21]:
metrics.r2_score(y_test,y_pred)

0.864107207815621

In [22]:
y_pred = y_pred.reshape(y_pred.shape[0])

In [23]:
y_pred = (y_test_max-y_test_min) * y_pred + y_test_min

In [40]:
y_pred.shape

(1459,)

In [43]:
result_df = pd.DataFrame({"Id":np.linspace(y_pred.shape[0]+2,(y_pred.shape[0]+1+y_pred.shape[0]),y_pred.shape[0],dtype=int), "SalePrice":y_pred.astype(np.int)})

In [44]:
result_df.to_csv("results.csv",index=False)

In [46]:
model.save('./weights/house_prices')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./weights/house_prices/assets
